Allen Institute resources for API access:
- http://help.brain-map.org/display/api/Allen+Brain+Atlas+API
- http://help.brain-map.org/display/api/Downloading+3-D+Expression+Grid+Data
- http://help.brain-map.org/display/api/Example+Queries+for+Experiment+Metadata
- http://help.brain-map.org/pages/viewpage.action?pageId=5308449

XML parsing:
- https://www.datacamp.com/community/tutorials/python-xml-elementtree
- https://www.kite.com/python/answers/how-to-download-a-csv-file-from-a-url-in-python


In [87]:
import os
import pandas as pd
import requests
# import xml.etree.ElementTree as ET

In [57]:
os.getcwd()

'/projects/abeauchamp/Projects/MouseHumanMapping/Paper_TranscriptomicSimilarity/AllenMouseBrainAtlas'

In [70]:
data_dir = os.getcwd()+'/Data/'

In [79]:
plane = 'coronal'
# plane = 'sagittal'

In [80]:
metadata_file = 'AMBA_metadata_{}.csv'.format(plane)
if os.path.isfile(data_dir+metadata_file):
    dfMetadata = pd.read_csv(data_dir+metadata_file)
else:
    abi_query_metadata = "http://api.brain-map.org/api/v2/data/SectionDataSet/query.csv?"+\
"criteria=[failed$eqfalse],plane_of_section[name$eq{}],products[abbreviation$eqMouse],treatments[name$eqISH],genes&".format(plane)+\
"tabular=data_sets.id+as+experiment_id,data_sets.section_thickness,data_sets.specimen_id,"+\
"plane_of_sections.name+as+plane,"+\
"genes.acronym+as+gene,genes.name+as+gene_name,genes.chromosome_id,genes.entrez_id,genes.genomic_reference_update_id,genes.homologene_id,genes.organism_id&"+\
"start_row=0&num_rows=all"
    
    dfMetadata = pd.read_csv(abi_query_metadata)
    dfMetadata.to_csv(data_dir+metadata_file)

In [81]:
dfMetadata.shape

(4345, 12)

In [95]:
temp_id = dfMetadata.loc[0,'experiment_id']

In [96]:
url = 'http://api.brain-map.org/grid_data/download/{}'.format(temp_id)
url

'http://api.brain-map.org/grid_data/download/112646890'

In [97]:
r = requests.get(url)


In [98]:
outfile = 'Data/Expression/temp.zip'
with open(outfile, 'wb') as file:
    file.write(r.content)

In [99]:
from zipfile import ZipFile
expression_dir = 'Data/Expression/'
raw_file = 'energy.raw'
minc_file = 'energy.mnc'

In [100]:
with ZipFile(expression_dir+'temp.zip', 'r') as file:
    file.extract(raw_file, path = expression_dir)

In [104]:
cmd = 'cat {} | rawtominc {} -signed -float -ounsigned -oshort -xstep 0.2 -ystep 0.2 -zstep 0.2 -clobber 58 41 67'.format(expression_dir+raw_file,expression_dir+minc_file)
cmd

'cat Data/Expression/energy.raw | rawtominc Data/Expression/energy.mnc -signed -float -ounsigned -oshort -xstep 0.2 -ystep 0.2 -zstep 0.2 -clobber 58 41 67'

In [105]:
os.system(cmd)

sh: 1: rawtominc: not found
cat: write error: Broken pipe


32512